# <center>Machine learning from scratch - Part II</center>
## <center>WebValley reimagined 2020</center>
### <center>Marco Chierici</center>
#### <center>FBK/MPBA</center>

In this handout we will go through basic concepts of machine learning using Python and scikit-learn on a real-world dataset of biological relevance [Zhang et al, _Genome Biology_ , 2015].

# Neuroblastoma dataset

In this section, we will focus on the SEQC Neuroblastoma dataset, specifically on **a subset of 272 samples (136 training, 136 test)**, aiming at predicting an **extreme disease outcome** (favorable vs unfavorable samples: see [main paper](https://www.ncbi.nlm.nih.gov/pubmed/26109056)).

<img src="images/zhang.png" width="65%" />

<img src="images/zhang_tab2.png" width="95%" />

The data was preprocessed a bit to facilitate the progress of the tutorial.

Let's start by loading a few modules that we'll be using later:

In [ ]:
import numpy as np
import pylab as plt ## for plotting
import pandas as pd ## for reading text files and manipulating data frames
from sklearn import neighbors ## kNN classifier
from pathlib import Path ## for creating paths in a neat way
np.random.seed(42) ## set random seed just in case

Define files to read:

In [ ]:
##  for convenience, define the data directory as a variable
DATA_DIR = Path("data")

In [ ]:
DATA_TR = DATA_DIR / "MAV-G_272_tr.txt"
DATA_TS = DATA_DIR / "MAV-G_272_ts.txt"
LABS_TR = DATA_DIR / "labels_tr.txt"
LABS_TS = DATA_DIR / "labels_ts.txt"

In [ ]:
LABS_TS

_Note:_ from now on we will use the "tr" suffix to denote the training set, and "ts" for the test set.

Read the files in as _pandas dataframes_ (they are conceptually like R data frames):

In [ ]:
data_tr = pd.read_csv(DATA_TR, sep="\t")
data_ts = pd.read_csv(DATA_TS, sep="\t")

The function `read_csv` has a lot more input arguments to deal with different situations.

If you want to know more about this or any other Python function, use the `help(function_name)` command or `function_name?`.

Give a look at what we have here, start with getting the dimensions of what we just uploaded:

In [ ]:
data_tr.shape

What's inside?

A peek at the first rows reveals that the first column (the dataframe index) contains the sample IDs, and the remaining columns are genes:

In [ ]:
data_tr.head()

Drop the first column from the train and test expression sets, since it's just the sample IDs (we put them in to be able to check whether samples and labels match, but once we are sure of what we are doing we don't really need them anymore).

In [ ]:
data_tr = data_tr.drop('sampleID', axis=1)
data_ts = data_ts.drop('sampleID', axis=1)

Check what happened

In [ ]:
data_tr.head()

Read in the files containing label information and check how they look like.

In [ ]:
labs_tr = pd.read_csv(LABS_TR, sep = "\t")
labs_ts = pd.read_csv(LABS_TS, sep = "\t")
labs_tr.head()

We have to fit one model for each label type, so we need to select one column at a time. We start with CLASS, in principle we could consider looping over the columns of interest. In this case no need to remove the first column, since we are using one column at a time.

In [ ]:
class_lab_tr = labs_tr[['CLASS']]
class_lab_ts = labs_ts[['CLASS']]
## give a look at one of the two
class_lab_tr.head()

For the remaining part of this hands-on, we need the data and labels to be stored in Numpy arrays (remember the `.values` function?):

In [ ]:
x_tr = data_tr.values
x_ts = data_ts.values

y_tr = class_lab_tr.values.ravel()
y_ts = class_lab_ts.values.ravel()

In [ ]:
y_tr

In [ ]:
y_tr.shape



---


*Naming conventions: in the machine learning world, usually `x` is the data and `y` the target variable (the labels). *

---



When coding, it is a good practice to have a peek at the resulting variables, to be sure everything is OK: i.e., is that variable like it is supposed to be? Did I accidentally throw away a feature column?

This can avoid lots of problems later on!

In [ ]:
x_tr.shape

Let's go back to the sample labels:

In [ ]:
y_tr.shape

---

### Quick recap

- class_lab_tr = 0 indicates **unfavorable** neuroblastoma samples (**bad** outcome)
- class_lab_tr = 1 indicates **favorable** neuroblastoma samples (**good** outcome)

---

# Data preprocessing

The downstream analysis can benefit from data preprocessing, i.e., rescaling or standardizing data values.

In Scikit learn you can use `MinMaxScaler` or `StandardScaler` in the `preprocessing` submodule. Here is an example using the `MinMaxScaler`:

In [ ]:
from sklearn.preprocessing import MinMaxScaler
## first you need to create a "scaler" object
scaler = MinMaxScaler(feature_range=(-1,1))
## then you actually scale data by fitting the scaler object on the data
scaler.fit(x_tr)
x_tr = scaler.transform(x_tr)
x_ts = scaler.transform(x_ts)

Note how we transformed the test set: we computed the scaling parameters on the training set and applied them to the test set. In this way, we did not use any information in the test set to standardize it.

# Data exploration

Remember the scatterplot matrix we showed on the 4-feature Iris data?

_How should we visualize feature relationships on the neuroblastoma dataset with 50K features? Would it be a good idea to compute a scatterplot matrix?_

It is probably better to first reduce the dimensionality of our data.

Principal Component Analysis (PCA) is one example of data dimesionality reduction technique. It finds a sequence of linear combination of the variables (called the _principal components_) that explain the maximum variance and summarize the most information in the data and are mutually uncorrelated.

Let's perform an **unsupervised learning** task on our data set "as is" by decomposing it in its Principal Components.

In scikit-learn, we can use the module PCA:

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)

So far we have a PCA _object_ but no transformation yet.

To actually transform the data, we'll have to _fit_ the PCA object on the training data, and then _transforming_ them in the Principal Component space:

In [ ]:
pca.fit(x_tr)
z_tr = pca.transform(x_tr)

In [ ]:
z_tr.shape

In [ ]:
x_tr.shape

Let's have a look at the _variance ratio_, i.e. the percentage of the variance explained by each component:

In [ ]:
print(pca.explained_variance_ratio_)

_What can you understand from these variance percentages? Could this task be "predictable" by some sort of model?_

Is it always convenient to visualize the first two principal components in a scatterplot, in order to get a first assessment of the goodness of the decomposition.

We will color the points in the plot according to our sample labels.

In [ ]:
f = plt.figure()
plt.scatter(z_tr[y_tr == 0, 0], z_tr[y_tr == 0, 1], color="r")
plt.scatter(z_tr[y_tr == 1, 0], z_tr[y_tr == 1, 1], color="b")
plt.title("PCA of Train data")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()
f.savefig("PCA_train.pdf")

_Now apply the transformation to the test data, plot it, and save it as PDF._

In [ ]:
## exercise here

# method 1: PCA on the test set
pca.fit(x_ts)
z_ts = pca.transform(x_ts)

# method 2: PCA on the train, use it to transform the test set
pca.fit(x_tr)
z_ts = pca.transform(x_ts)


We now perform a UMAP transformation on the data, recalling what we did on the Iris dataset. Notice how we now split the `fit` and the `transform` operations:

In [ ]:
import umap
reducer = umap.UMAP(random_state=11)
reducer.fit(x_tr)
embedding = reducer.transform(x_tr)
embedding.shape

In [ ]:
df_2D = pd.DataFrame(embedding,
                     columns=['UMAP1', 'UMAP2'])
df_2D['class'] = y_tr
df_2D.head()

In [ ]:
for key, group in df_2D.groupby(['class']):
    plt.plot(group.UMAP1, group.UMAP2, 'o', alpha=0.7, label=key)
    
plt.legend(loc=0, fontsize=12)

Once we have a UMAP object projecting the training set into a low-dimensional space, we can project the test set onto the same manifold. 

We use the `transform` method on the reducer object, and plot the resulting transformation:

In [ ]:
embedding_ts = reducer.transform(x_ts)
df_2D = pd.DataFrame(embedding_ts,
                     columns=['UMAP1', 'UMAP2'])
df_2D['class'] = y_ts

for key, group in df_2D.groupby(['class']):
    plt.plot(group.UMAP1, group.UMAP2, 'o', alpha=0.7, label=key)
    
plt.legend(loc=0, fontsize=12)


## Supervised Learning

### k-NN classifier

Based on the PCA we built on our data, we decide to try some supervised learning on them.

Scikit-learn provides you access to several models via a very convenient _fit_ and _predict_ interface.

For example, let's fit a **k-NN** model on the whole training data and then use it to predict the labels of the test data.

In [ ]:
knn = neighbors.KNeighborsClassifier(n_neighbors=10)

In [ ]:
knn.fit(x_tr, y_tr)

In [ ]:
y_pred_knn = knn.predict(x_ts) # predict labels on test data

_In general, a classifier has **parameters** that need to be tuned. Default choices are not good in all situations._

_For example, in k-NN the main parameter is the **number of neighbors** used in the nearest neighbors algorithm._

_More on this in the next lecture!_

To evaluate the predictions we need some kind of metrics. 

_Exercise: extract confusion matrix and try calculate metrics by hand._

### Recap: confusion matrix

In this example, the first row is class 0, so the confusion matrix will look like:

|      |  |  Predicted  |    |
|------|-----------|----|----|
|      |           | 0 | 1  |
| True | 0        | TN | FP |
|      | 1         | FN | TP |


In [ ]:
from sklearn.metrics import confusion_matrix
conf = confusion_matrix(y_ts, y_pred_knn)
conf

The total number of class 0 test samples (AN = All Negatives) should be equal to the sum of the first row of the confusion matrix, i.e., TN + FP:

In [ ]:
np.sum(y_ts==0) # total number of "class 0" samples in the test set

Similarly for class 1, i.e., AP = All Positives = TP + FN:

In [ ]:
np.sum(y_ts==1) # total number of "class 1" samples in the test set

Compute the Accuracy, remembering/using the formula: 

ACC = (TN + TP) / (TN + TP + FN + FP)

In [ ]:
tp = conf[1,1]
tn = conf[0,0]
fp = conf[0,1]
fn = conf[1,0]

acc = (tn + tp) / (tn + tp + fn + fp)
print(acc)

Now compute the Sensitivity.

Remember the formula:

SENS = TP / (TP + FN)

In [ ]:
tp / (tp + fn)

Computing metrics by hand is good, but what about a quicker option?

As seen in the lectures, Scikit Learn offers a handy broad range of functions to evaluate your classifier through its submodule `metrics`.

Let's compute the accuracy using the scikit-learn built-in function `accuracy_score`, taking as input the predicted labels (`y_pred_knn`) and the true labels (`y_ts`):

`accuracy_score(y_ts, y_pred_knn)`

In [ ]:
from sklearn import metrics
metrics.accuracy_score(y_ts, y_pred_knn)

What about Sensitivity? The built-in function is called `recall_score`, as Recall is an alternate name for Sensitivity. Again, its input are the predicted and the true labels.


In [ ]:
metrics.recall_score(y_ts, y_pred_knn)

Scikit-learn also provides a neat `metrics.classification_report` function that outputs a few metrics stratified by class:

In [ ]:
print(metrics.classification_report(y_ts, y_pred_knn))

Let's consider the Matthews Correlation Coefficient (MCC):

![MCC formula](https://www.researchgate.net/profile/Pablo_Moscato/publication/223966631/figure/fig1/AS:305103086080001@1449753652505/Calculation-of-Matthews-Correlation-Coefficient-MCC-A-Contingency-matrix_W640.jpg)

*Q: Do you remember the main features of MCC?*

In scikit-learn it is computed by the `metrics.matthews_corrcoef` function.

If we get the MCC for our kNN predictions, we can observe that it is in line with our *a priori* knowledge of the dataset (from the article):

In [ ]:
print(metrics.matthews_corrcoef(y_ts, y_pred_knn))

*Compare the metrics that you computed so far. What can you say about this classification task? Does the classifier learn something?*

The metrics may look good (e.g., accuracy around 0.8, MCC above 0.6) but...

... how do you know if this model performs similarly well on unseen data?

In other words, does this model *generalize* beyond its training set?

This is why *data partitioning* techniques are used.

## Data partitioning

### Hold-out strategy

The idea behind data partitioning is to split your original data set into a **train** portion (for developing your machine learning model) and a **test** portion (for evaluating the performance of the trained model).

The simplest and most straightforward way to partition your data set is to randomly split it in two groups (*hold-out strategy*).


---


"But we already have a dataset split into train and test!", you may object.

Well, the data was previously split into balanced train and test sets of 136 samples each. This is somewhat different from the 80/20 train/test split we mentioned in the lecture. In fact, this specific data split was created ad-hoc during the article preparation to ensure balance in the various clinical characteristics of the neuroblastoma samples represented (e.g. MYCN amplification status, INSS tumor staging, … Full details are reported in the article).

For the sake of this tutorial, we will further split the neuroblastoma train set into two subsets.


---


You achieve this using scikit-learn's function `train_test_split`, in the `model_selection` submodule.

For example, let's split the data (x_tr) into 80% train and 20% test (note the argument `test_size=0.2`)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_tr, y_tr, test_size=0.2, random_state=1)



---

What is the random_state?

Whenever randomness is involved in a computer program, we need to rely on some sort of workaround because computers follow their instructions blindly and they are therefore completely predictable.

One approach relies on *Pseudo-Random Number Generators* (PRNGs). 

PNRGs are algorithms that use mathematical formulas or precalculated tables to produce sequences of numbers that appear random. 

PNRGs are initialized by a *seed* (an integer), so that *the same seed yields the same sequence of pseudo-random numbers*. This is useful for reproduciblity.


---



*Now, retrain a kNN model on X_train and evaluate its performance on X_test. Try using different random states for data splitting.*

In [ ]:
from sklearn import metrics
knn = neighbors.KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

acc = metrics.accuracy_score(y_test, y_pred_knn)
mcc = metrics.matthews_corrcoef(y_test, y_pred_knn)

print("Accuracy = " , acc)
print("MCC = ", mcc)

## Homework

* Use the other labels ("SEX", "UK") to train a classifier
* Fit on the "tr" set and predict on "ts"
* Use the hold-out strategy
* Evaluate the performance for each type of label

## What next?

* Cross-validation
* Other classifiers: Support vector machines, Random Forests, Neural nets
* Feature ranking
* Parameter tuning